# plotting healpix data with xdggs and lonboard

For this to work, we need:
- `xarray`
- `numpy`
- `xdggs`
- `lonboard`
- `arro3-core`
- `matplotlib`

All of these can be installed using the `explore` extra:

In [1]:
#!pip install "xdggs[explore] @ git+https://github.com/keewis/xdggs.git#branch=lonboard"

!pip install "xdggs[explore] @ git+https://github.com/keewis/xdggs.git@lonboard"

  Cloning https://github.com/keewis/xdggs.git to /tmp/pip-install-ly7q_fzp/xdggs_298a07348ea54afd8df1bd6107f14d8e
  Running command git clone --filter=blob:none --quiet https://github.com/keewis/xdggs.git /tmp/pip-install-ly7q_fzp/xdggs_298a07348ea54afd8df1bd6107f14d8e
  Resolved https://github.com/keewis/xdggs.git to commit 775c76be55c661cc8ceb5914fedefda0db5edb89
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 53.0 MB/s eta 0:00:00
  Created wheel for xdggs: filename=xdggs-0.1.dev53+g775c76b-py3-none-any.whl size=23214 sha256=21f98f3d1d2336b67f7713b979d5c33f63b274568277dc2298ebcace3c570431
  Stored in directory: /tmp/pip-ephem-wheel-cache-pulnu04g/wheels/11/dd/6e/9fa09ae5bad1a9382b041ca100a1fcec2819ecccbd89598f51
Successfully built xdggs


imports

In [1]:
import numpy as np
import xarray as xr

import xdggs

generate the test dataset

In [5]:
level = 8
cell_ids = np.arange(12 * 4**level)
grid_info = {"grid_name": "healpix", "level": level, "indexing_scheme": "nested"}

ds = (
    xr.Dataset(coords={"cell_ids": ("cells", cell_ids, grid_info)})
    .pipe(xdggs.decode)
    .pipe(lambda ds: ds.merge(ds.dggs.cell_centers()))
    .assign(data=lambda ds: np.cos(6 * np.radians(ds["latitude"])) * np.sin(6 * np.radians(ds["longitude"])))
)
ds

<xarray.Dataset> Size: 25MB
Dimensions:    (cells: 786432)
Coordinates:
  * cell_ids   (cells) int64 6MB 0 1 2 3 4 ... 786428 786429 786430 786431
    latitude   (cells) float64 6MB 0.1492 0.2984 0.2984 ... -0.2984 -0.1492
    longitude  (cells) float64 6MB 45.0 45.18 44.82 45.0 ... 315.2 314.8 315.0
Dimensions without coordinates: cells
Data variables:
    data       (cells) float64 6MB -0.9999 -0.9993 -0.9993 ... 0.9993 0.9999
Indexes:
    cell_ids  HealpixIndex(nside=8, indexing_scheme=nested, rotation=[0.0, 0.0])

create the map and visualize

In [8]:
ds["data"].dggs.explore(center=0, cmap="coolwarm",alpha=0.5)

Map(layers=[SolidPolygonLayer(filled=True, get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x7f1884da…

In [9]:
import xarray as xr

ds = xr.open_dataset(
    "https://eerie.cloud.dkrz.de/datasets/nextgems.IFS_9-FESOM_5-production.2D_hourly_healpix512/zarr",
    engine="zarr",
    chunks={},
    consolidated=True,
)
ds

<xarray.Dataset> Size: 383TB
Dimensions:  (time: 262537, value: 3145728)
Coordinates:
    lat      (value) float64 25MB dask.array<chunksize=(3145728,), meta=np.ndarray>
    lon      (value) float64 25MB dask.array<chunksize=(3145728,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2MB 2020-01-20 ... 2050-01-01
Dimensions without coordinates: value
Data variables: (12/58)
    10si     (time, value) float64 7TB dask.array<chunksize=(3, 3145728), meta=np.ndarray>
    10u      (time, value) float64 7TB dask.array<chunksize=(3, 3145728), meta=np.ndarray>
    10v      (time, value) float64 7TB dask.array<chunksize=(3, 3145728), meta=np.ndarray>
    2d       (time, value) float64 7TB dask.array<chunksize=(3, 3145728), meta=np.ndarray>
    2t       (time, value) float64 7TB dask.array<chunksize=(3, 3145728), meta=np.ndarray>
    blh      (time, value) float64 7TB dask.array<chunksize=(3, 3145728), meta=np.ndarray>
    ...       ...
    tp       (time, value) float64 7TB dask.array<chunksize=(3, 3145728), meta=np.ndarray>
    tprate   (time, value) float64 7TB dask.array<chunksize=(3, 3145728), meta=np.ndarray>
    tsr      (time, value) float64 7TB dask.array<chunksize=(3, 3145728), meta=np.ndarray>
    tsrc     (time, value) float64 7TB dask.array<chunksize=(3, 3145728), meta=np.ndarray>
    ttr      (time, value) float64 7TB dask.array<chunksize=(3, 3145728), meta=np.ndarray>
    ttrc     (time, value) float64 7TB dask.array<chunksize=(3, 3145728), meta=np.ndarray>
Attributes: (12/13)
    edition:            2
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          1003
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...
    title:              nextgems.IFS_9-FESOM_5-production.2D_hourly_healpix512
    ...                 ...
    time_min:           2020-01-20T00:00:00.000000000
    time_max:           2050-01-01T00:00:00.000000000
    creation_date:      2024-10-08T06:15:02Z
    authors:            Wieners, Karl-Hermann (MPIMet), Aguridan, Razvan (ECM...
    contact:            Wieners, Karl-Hermann (MPIMet), Rackow, Thomas (ECMWF)
    license:            Creative Commons Attribution 4.0 International (CC BY...

In [16]:
data=ds["msl"].isel(time=1000).compute().assign_coords(cell_ids=('value', np.arange(12*4**9 ), {"grid_name":"healpix","level": 9, "indexing_scheme": "nested"})).pipe(xdggs.decode)
data

<xarray.DataArray 'msl' (value: 3145728)> Size: 25MB
array([100984.51660156, 100988.79394531, 100979.44824219, ...,
       101204.87207031, 101202.56152344, 101200.22753906])
Coordinates:
    lat       (value) float64 25MB 0.0746 0.1492 0.1492 ... -0.1492 -0.0746
    lon       (value) float64 25MB 45.0 45.09 44.91 45.0 ... 315.1 314.9 315.0
    time      datetime64[ns] 8B 2020-03-01T16:00:00
  * cell_ids  (value) int64 25MB 0 1 2 3 4 ... 3145724 3145725 3145726 3145727
Dimensions without coordinates: value
Indexes:
    cell_ids  HealpixIndex(nside=9, indexing_scheme=nested, rotation=[0.0, 0.0])
Attributes: (12/15)
    paramId:                    151
    dataType:                   fc
    numberOfPoints:             3145728
    typeOfLevel:                meanSea
    stepUnits:                  1
    stepType:                   instant
    ...                         ...
    name:                       Mean sea level pressure
    cfName:                     air_pressure_at_mean_sea_level
    cfVarName:                  msl
    missingValue:               9999
    NV:                         0
    gridDefinitionDescription:  150

In [19]:
data.dggs.explore(#center=0, 
                  cmap="viridis"
    ,alpha=0.8)

Map(layers=[SolidPolygonLayer(filled=True, get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x7f185096…

In [23]:
#data.dggs.sel_latlon(latitude=slice(30,50), longitude=slice(0,10))

data.sel(cell_ids=slice(0,1000)).dggs.explore(#center=0, 
                  cmap="viridis"
    ,alpha=0.8)

Map(layers=[SolidPolygonLayer(filled=True, get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x7f184fb0…